# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Output File (CSV)
output_data_file = "../output-data/cities.csv"

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv(output_data_file)
df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ambon,-3.70,128.18,24.00,94,40,4.10,ID,1603033212
1,hermanus,-34.42,19.23,20.00,61,41,2.24,ZA,1603033214
2,smithers,54.78,-127.17,0.00,93,90,2.10,CA,1603033188
3,menongue,-14.66,17.69,34.76,12,0,3.39,AO,1603033216
4,gebre guracha,9.80,38.40,14.31,80,57,0.52,ET,1603033218


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

locations = df[["Lat", "Lng"]].astype("float")

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=df["Humidity"], 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
hotel_df = df.loc[(df["Max Temp"]<27) & (df["Max Temp"]>20) & (df["Cloudiness"]<20) & (df["Humidity"]<60) & (df["Wind Speed"]<4)]
hotel_df.reset_index(drop=True)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,khani,41.96,42.96,23.00,56,1,2.10,GE,1603033419
1,chuy,-33.70,-53.46,23.94,56,0,2.98,UY,1603033432
2,dehloran,32.69,47.27,24.23,18,0,2.03,IR,1603033499
3,naica,27.85,-105.48,22.55,34,0,1.72,MX,1603033761
4,viedma,-40.81,-63.00,23.33,26,5,2.68,AR,1603034007


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df["Hotel Name"] = ""

target_search = "hotel"
target_type = "lodging"
radius = 5000

for index, row in hotel_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    target_coordinates = f"{row['Lat']}, {row['Lng']}"

    params = {
            "location": target_coordinates,
            #"keyword": target_search,
            "radius": radius,
            "type": target_type,
            "key": g_key
        }
        # make request
    hotels = requests.get(base_url, params=params).json()
    hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
    
hotel_df.reset_index(drop=True)

C:\Users\Elliott\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Elliott\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,khani,41.96,42.96,23.00,56,1,2.10,GE,1603033419,კურორტი ზეკარი/Resort Zekari
1,chuy,-33.70,-53.46,23.94,56,0,2.98,UY,1603033432,Nuevo Hotel Plaza
2,dehloran,32.69,47.27,24.23,18,0,2.03,IR,1603033499,مسافرخانه قدس(عبدالله خداکرمی)
3,naica,27.85,-105.48,22.55,34,0,1.72,MX,1603033761,Departamentos Nicole
4,viedma,-40.81,-63.00,23.33,26,5,2.68,AR,1603034007,Hotel Cristal


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))